In [ ]:
import os
import sys
import pandas as pd

# setting warnings
import warnings
warnings.simplefilter(action='ignore', category = Warning)

# add parent folder path to the namespace
sys.path.append(os.path.dirname(os.getcwd()))

# import modules and components
from utils.data_assets import DataGenerator, TensorDataSet
from utils.model_assets import ModelValidation, Inference
import utils.global_paths as globpt
import configurations as cnf

# specify relative paths from global paths and create subfolders
cp_path = os.path.join(globpt.train_path, 'checkpoints')
os.mkdir(cp_path) if not os.path.exists(cp_path) else None

# Load and prepare data

In [ ]:
# load the model for inference and print summary
inference = Inference(cnf.seed) 
model, parameters = inference.load_pretrained_model(cp_path)
model_path = inference.folder_path
model.summary(expand_nested=True)

# load data
filepath = os.path.join(model_path, 'preprocessing', 'train_data.csv')                
train_data = pd.read_csv(filepath, sep=';', encoding='utf-8')
filepath = os.path.join(model_path, 'preprocessing', 'test_data.csv')                
test_data = pd.read_csv(filepath, sep=';', encoding='utf-8')

# initialize the images generator for the train and test data, and create the 
# tf.dataset according to batch shapes
train_generator = DataGenerator(train_data, 20, cnf.picture_shape, 
                                augmentation=False, shuffle=True)
test_generator = DataGenerator(test_data, 20, cnf.picture_shape, 
                               augmentation=False, shuffle=True)

# initialize the TensorDataSet class with the generator instances
# create the tf.datasets using the previously initialized generators 
datamaker = TensorDataSet()
train_dataset = datamaker.create_tf_dataset(train_generator)
test_dataset = datamaker.create_tf_dataset(test_generator)

# 1. Model validation

## 1.1 Evaluate model loss and accuracy

In [ ]:
validator = ModelValidation(model)

# create subfolder for evaluation data
eval_path = os.path.join(model_path, 'evaluation') 
os.mkdir(eval_path) if not os.path.exists(eval_path) else None

# evluate the model on both the train and test dataset
train_eval = model.evaluate(train_dataset, batch_size=20, verbose=1)
test_eval = model.evaluate(test_dataset, batch_size=20, verbose=1)

print(f'''  
Train dataset:
- Loss:   {train_eval[0]}
- Metric: {train_eval[1]} 

Test dataset:
- Loss:   {test_eval[0]}
- Metric: {test_eval[1]}        
''')

## 1.2 Visual validation 

In [ ]:
# perform visual validation for the train dataset (initialize a validation tf.dataset
# with batch size of 10 images)
validation_batch = train_dataset.unbatch().batch(10).take(1)
for images, labels in validation_batch:
    recostructed_images = model.predict(images, verbose=0)
    validator.visual_validation(images, recostructed_images, 'visual_validation_train', 
                                eval_path)

# perform visual validation for the test dataset (initialize a validation tf.dataset
# with batch size of 10 images)
validation_batch = test_dataset.unbatch().batch(10).take(1)
for images, labels in validation_batch:
    recostructed_images = model.predict(images, verbose=0) 
    validator.visual_validation(images, recostructed_images, 'visual_validation_test',
                                eval_path)